In [1]:
import pandas as pd
import os
import glob
import numpy as np # type: ignore
import re
from pandarallel import pandarallel
from textblob import TextBlob

df = pd.read_csv('/Users/allanleung/Downloads/killarney_winter_test.csv', encoding='latin1')


In [2]:
# Drop Garbage Column 

df = df.drop(columns=['ChildSeasonName'])
df = df.drop(columns=['NumberEnrolled'])
df = df.drop(columns=['Type'])
df = df.drop(columns=['EnrollMin'])
df = df.drop(columns=['EnrollMax'])
df = df.drop(columns=['RiskCategoryName'])
df = df.drop(columns=['GradeMin'])
df = df.drop(columns=['GradeMax'])
df = df.drop(columns=['NumberOfHours'])
df = df.drop(columns=['NumberOfDates'])
df = df.drop(columns=['FeeDescription'])
df = df.drop(columns=['AccountName'])
df = df.drop(columns=['KeyFeesTotal'])
df = df.drop(columns=['OtherFeesTotal'])
df = df.drop(columns=['FeeSummary'])


In [3]:
# Data Clean Up

# other_category_column = df["OtherCategoryName"]
# Clean the column: Remove numbers, hyphens, and extra spaces
df['OtherCategoryName'] = df['OtherCategoryName'].str.replace(r'^\d+\s*-\s*', '', regex=True).str.strip()


# Define keywords to identify music or vocals related activities
music_keywords = ['music', 'vocal', 'choir', 'sing', 'piano', 'guitar', 'ukulele', 'violin', 'fiddle']

# Update CategoryName for rows where ActivityName contains music-related keywords
df['CategoryName'] = df.apply(
    lambda row: 'Music & Vocals' if any(keyword.lower() in str(row['ActivityName']).lower() for keyword in music_keywords) else row['CategoryName'],
    axis=1
)

# Define keywords to identify dance-related activities
dance_keywords = ['dance', 'ballet', 'hip hop', 'jazz', 'zumba', 'line dancing', 'tap', 'kpop', 'breakers', 'ballerinas']

# Update CategoryName for rows where ActivityName contains dance-related keywords
df['CategoryName'] = df.apply(
    lambda row: 'Dance' if any(keyword.lower() in str(row['ActivityName']).lower() for keyword in dance_keywords) else row['CategoryName'],
    axis=1
)

# Update 'CategoryName' column names
df['CategoryName'] = df['CategoryName'].replace("Art, Culture & Environment", "Art")
df['CategoryName'] = df['CategoryName'].replace("Volunteer Opportunities", "Workshop")
df['CategoryName'] = df['CategoryName'].replace("Youth Leadership", "Workshop")
df['CategoryName'] = df['CategoryName'].replace("Food, Cooking and Gardening", "Culinary Arts")
df['CategoryName'] = df['CategoryName'].replace("Outings & Bus Excursions", "Social")

# Overwrite entire column with "Winter-2025"
for column in df.columns:
    if "SeasonName" in column:
        df[column] = "Winter-2025"

# Remove "at *Killarney Community Centre" from every row in ActivityLocation
df['ActivityLocation'] = df['ActivityLocation'].str.replace(r'at \*Killarney Community Centre', '', regex=True).str.strip()


# Convert the BeginningDate column to datetime, assuming format "dd-MMM" (day and abbreviated month)
df['BeginningDate'] = pd.to_datetime(df['BeginningDate'], format='%d-%b', errors='coerce')

# Update the year to 2025 while keeping the original day and month
df['BeginningDate'] = df['BeginningDate'].apply(lambda x: x.replace(year=2025) if pd.notnull(x) else x)

# EndDate
df['EndingDate'] = pd.to_datetime(df['EndingDate'], format='%d-%b', errors='coerce')

# Update the year to 2025 while keeping the original day and month
df['EndingDate'] = df['EndingDate'].apply(lambda x: x.replace(year=2025) if pd.notnull(x) else x)



In [4]:
# Create the AgeRange Column

df['AgeRange'] = np.where(
    (df['AgesMin'] == 0) & (df['AgesMax'] == 0),  # Edge Case 1: AgesMin and AgesMax are 0
    df['AgesMinMonth'].astype(str) + " - " + df['AgesMaxMonth'].astype(str) + " months",
    np.where(
        (df['AgesMin'] == 0) & (df['AgesMax'] == 1) & (df['AgesMinMonth'] > 0),  # Edge Case 2: AgesMin = 0, AgesMax = 1
        df['AgesMinMonth'].astype(str) + " - 12 months",
        np.where(
            df['AgesMin'] == 65,  # Condition 1: AgesMin is exactly 65
            "65+",
            np.where(
                (df['AgesMin'] > 0) & (df['AgesMax'] == 0),  # Condition 2: AgesMin > 0 and AgesMax == 0
                df['AgesMin'].astype(str) + "+",
                np.where(
                    df['AgesMax'] == 100,  # Condition 3: AgesMax is 100
                    df['AgesMin'].astype(str) + "+",
                    df['AgesMin'].astype(str) + " - " + df['AgesMax'].astype(str) + " yrs"  # Default case
                )
            )
        )
    )
)

# Function to clean and format age ranges in ActivityName
def clean_activity_name(title):
    # Add spaces around the hyphen and "yrs"
    return re.sub(r'(\d+)\s*-\s*(\d+)\s*yrs', r'\1 - \2 yrs', title)

# Apply the cleaning function to the ActivityName column
df['ActivityName'] = df['ActivityName'].apply(clean_activity_name)


# Define a function to map and clean weekdays
def clean_weekdays(day):
    if day.strip() == "M Tu W Th F":  # Handle Monday to Friday case
        return "Monday to Friday"
    elif day.strip() == "Every day":  # Handle "Every day"
        return "Every day"
    else:
        # Split the days and map to their full abbreviations
        days_map = {
            "M": "Mon",
            "Tu": "Tue",
            "W": "Wed",
            "Th": "Thu",
            "F": "Fri",
            "Sa": "Sat",
            "Su": "Sun"
        }
        # Join the cleaned abbreviations
        return " ".join([days_map.get(d, d) for d in day.split()])

# Apply the function to the WeekDays column
df['WeekDays'] = df['WeekDays'].apply(clean_weekdays)


# Find the index of the 'Weekdays' column
weekdays_index = df.columns.get_loc('WeekDays')

# Reorder the columns: Insert 'AgeRange' before 'Weekdays'
columns = df.columns.tolist()
columns.insert(weekdays_index, columns.pop(columns.index('AgeRange')))
df = df[columns]




In [5]:
# Function to determine time of day
def determine_time_of_day(time_str):
    # Convert time string to datetime
    time = pd.to_datetime(time_str, format='%I:%M %p').time()
    
    if time >= pd.to_datetime('6:00 AM', format='%I:%M %p').time() and time < pd.to_datetime('12:00 PM', format='%I:%M %p').time():
        return 'Morning'
    elif time >= pd.to_datetime('12:00 PM', format='%I:%M %p').time() and time <= pd.to_datetime('5:00 PM', format='%I:%M %p').time():
        return 'Afternoon'
    else:
        return 'Evening'

# Apply the function to the DataFrame
df['TimeOfDay'] = df['StartingTime'].apply(determine_time_of_day)

# Find the index of the 'StartingTime' column
starting_time_index = df.columns.get_loc('StartingTime')

# Reorder the columns: Insert 'TimeOfDay' before 'StartingTime'
columns = df.columns.tolist()
columns.insert(starting_time_index, columns.pop(columns.index('TimeOfDay')))
df = df[columns]



In [6]:
# Function to generate the next versioned filename
def get_next_filename(filepath):
    directory, original_filename = os.path.split(filepath)
    filename, ext = os.path.splitext(original_filename)
    
    # Get all files in the directory that match the base filename pattern
    existing_files = glob.glob(os.path.join(directory, f"{filename}_*{ext}"))
    
    # Extract the highest version number
    max_version = 0
    for file in existing_files:
        # Extract the version number from the filename
        base, version = '_'.join(file.split('_')[:-1]), file.split('_')[-1].replace(ext, '')
        if version.isdigit():
            max_version = max(max_version, int(version))
    
    # Generate the new filename
    new_filename = f"{filename}_{max_version + 1}{ext}"
    return os.path.join(directory, new_filename)

# Generate the new filename
new_filename = get_next_filename("/Users/allanleung/Downloads/killarney_winter_test.csv")

# Export the updated DataFrame to the new CSV file
df.to_csv(new_filename, index=False)

print(f"File saved as: {new_filename}")


File saved as: /Users/allanleung/Downloads/killarney_winter_test_43.csv
